<a href="https://colab.research.google.com/github/Alfred78w/Fine-tuning-Arabart-model/blob/main/SODJI_EL_HOUSSNI_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center><big><big><big><b>Data Challenge</b></big></big></big></center>
<center><big><big><big><b>Darija ➡️ MSA Translation</b></big></big></big></center>

<center><big><big><i> By:</i></big></big></center>

<center><big><big><i>🌟 EL HOUSSNI HOUSSAM 🌟</i></big></big></center>
<center><big><big><i>🌟 SODJI SOUROU ALFRED 🌟</i></big></big></center>












**<big><big>Task B: Data Augmentation**

**<big> Step 1: Using online ressources**


To enrich our training data for Darija to MSA translation, we will begin by sourcing  vocabulary and texts from online resources and databases.

These online resources will provide us with authentic Darija sentences paired with their corresponding MSA translations, enhancing the diversity and quality of our dataset.

Sites web of our selected data:

https://tajinequiparle.com/en/english-moroccan-arabic-dictionary/a/ (web scraping)

https://atlasculturalfoundation.org/wp-content/uploads/2019/02/darija-phrasebook.pdf  (Using Data-->Import from Web, function in Excel)

https://www.loecsen.com/en/vocabulary-arabic-moroccan   (Using Data-->Import from Web, function in Excel)

https://docs.google.com/spreadsheets/d/1JC6LY9MxHwlxDiNM_e9rPAMFJSY0tt6BuRPcNI1Bf-U/edit#gid=91538701


After finding the data online, we will compile all the vocabulary encountered from online sources into a clean Excel sheet, ensuring its usability for further data augmentation and model training.

**<big> Step 2 : Using LLMs**



##Web scraping
For database augmentation, one of our key techniques is to take a bilingual dictionary ``[English <---> Darija]`` from online data. On the [tajinequiparle](https://tajinequiparle.com/en/english-moroccan-arabic-dictionary/a/) site there is a bilingual dictionary
 English <---> Darija]`` which we used to augment our database. So we set up a web scraping algorithm with the library
 `BeautifulSoup` which allowed us to have a database containing words in `English` and their translation in `Moroccan dialect` of approximately 4K lines. This database named `words.csv` contains `Word` which is the word, `Nature` - the nature of the word, `darija_Translation` - the translation into Moroccan dialect in Latin script, `Arabic_Writing` - the writing of the word in Arabic script, `Meaning`, and `relating to` which are not useful enough in our case.

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv

def scrape_tajine_qui_parle(lettre):
    # Page URL
    url = "https://tajinequiparle.com/en/english-moroccan-arabic-dictionary/"+ lettre +"/"

    # Page content retrieval
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")

    # Searching for items with the class "glossary-listing"
    links = soup.find_all("a", class_="glossary-listing")

    # Initialisation of the list of URLs
    urls = []

    # Browse elements to extract links
    for link in links:
       # Get the URL
        href = link.get("href")
        # Add the URL to the list
        urls.append(href)

    # Display the URLs
    # print(urls)

    # Function to retrieve information about a word page
    def get_word_info(url):
        response = requests.get(url)
        soup = BeautifulSoup(response.text, "html.parser")

        # Retrieve the English word
        word = soup.find("h2").text.strip() if soup.find("h2") else ""

        # Retrieve meanings and their translations
        natures = []
        meanings = []
        arabic_translations = []
        arabic_writings = []

        for h3 in soup.find_all("h3"):
            if h3.find("em"):
                nature_info = h3.find('em').text.strip()
                natures.append(nature_info)

        if natures == []: natures.append('')

        for h5 in soup.find_all("h5"):
            arabic_text = h5.text.strip().split("[", 1)
            if len(arabic_text) > 1:
                arabic_writing = arabic_text[1].split("]")[0].strip()
            else:
                arabic_writing = ""  # Or any other default value if needed
            meaning_text = h5.text.strip().split("(", 1)
            if len(meaning_text) > 1:
                meaning = meaning_text[1].split(")")[0].strip()
            else:
                meaning = ""  # Or any other default value if needed
            arabic_translation = ""
            arabic_translation = h5.find('q').text.strip() if h5.find('q') else("")
            meanings.append(meaning)
            arabic_translations.append(arabic_translation)
            arabic_writings.append(arabic_writing)
        return word, natures, meanings, arabic_translations, arabic_writings

    nom_fichier_csv = "words.csv"

    # Open a CSV file to write data in append mode
    with open(nom_fichier_csv, 'a', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['Word', 'Nature', 'darija_Translation', 'Arabic_Writing', 'Meaning', 'relating to']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        # Loop through the URLs to retrieve information for each word
        for url in urls:
            word, natures, meanings, arabic_translations, arabic_writings = get_word_info("https://tajinequiparle.com" + url)
            # Write the information into the CSV file
            # print(word)
            for i in range(len(meanings)):
                meaning_split = meanings[i].split(" ", 1)
                if len(meaning_split) > 1:
                    meaning = meaning_split[1]
                else:
                    meaning = meanings[i]  # Use the complete meaning if no split is possible
                writer.writerow({'Word': word, 'Nature': natures[i if i < len(natures) else -1],
                                'darija_Translation': arabic_translations[i],
                                'Arabic_Writing': arabic_writings[i], 'Meaning': meaning,
                                'relating to': meaning_split[0]})

# scrape_tajine_qui_parle("a")

In [ ]:
nom_fichier_csv = "words.csv"
    # Open a CSV file to write data
with open(nom_fichier_csv, 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['Word', 'Nature', 'darija_Translation', 'Arabic_Writing', 'Meaning', 'relating to']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    # Write the column names in the CSV file
    writer.writeheader()

letters = [chr(i) for i in range(ord('y'), ord('z')+1)]

for letter in letters:
    scrape_tajine_qui_parle(letter)

##Data Preprocessing
The extracted data undergoes preprocessing to clean and format it according to project requirements. This involve handling missing values and reformatting data into json file.
Data cleaning is crucial to eliminate errors, inconsistent formats, and irrelevant information. In our case this includes
- Removing duplicates to avoid training bias.
- Removing missing values
- Correcting typos or grammatical errors in source and target texts.



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import urllib

urllib.request.urlretrieve("https://onedrive.live.com/download?resid=758EF4B4D89BDE45%214789&authkey=!ABCcTMoUFdERYYI", 'test-4000.json')

('test-4000.json', <http.client.HTTPMessage at 0x7f8d4d1b1660>)

In [ ]:
import pandas as pd

# Load the Excel file
excel_file = "/content/our_final_database.xlsx"
df = pd.read_excel(excel_file)

# Drop rows with missing values
df = df.dropna()

# Remove duplicate rows
df = df.drop_duplicates()

# Rename columns to English names
df.rename(columns={"Darija_Writing": "Darija", "Word": "English"}, inplace=True)

# Select only the English, Darija, and MSA columns
df = df[["English", "Darija","MSA"]]

# Convert boolean values to strings
df = df.applymap(lambda x: str(x) if isinstance(x, bool) else x)


In [ ]:
import json
import pandas as pd

translations = []
for idx, row in df.iterrows():
    translation_dict = {
        "translation": {
            "idx": idx,
            "English": row["English"],
            "Darija": row["Darija"],
            "MSA": row["MSA"]
        }
    }
    translations.append(translation_dict)
with open("train2.json", "w", encoding="utf-8") as json_file:
    for translation in translations:
        json.dump(translation, json_file, ensure_ascii=False)
        json_file.write("\n")


# Task C

## Model Architecture

# The model we employ in this assignment is 'Arabart,' an encoder-decoder model. We have opted for Arabart due to the following qualities:

1. **Linguistic Specialization**: Arabart is specifically engineered to comprehend and generate Arabic text, which encompasses familiarity with linguistic nuances, syntax, and vocabulary unique to the Arabic language. This renders it particularly suitable for tasks involving Arabic, unlike more generalized models.
2. **Dialect Support**: Models like Arabart are trained on a broad spectrum of Arabic texts, including various dialects. This means they are better equipped to handle the specifics of Darija, a Maghrebi Arabic dialect, when translating to or from Modern Standard Arabic (MSA).
3. **Translation Quality**: Thanks to its specialized training on Arabic texts, Arabart can provide superior translation quality for language pairs involving Arabic compared to non-specialized models. This includes better management of complex grammatical structures and more precise vocabulary selection.
4. **Adaptability**: Arabart, being a member of the BART family of models, is designed for text generation tasks, including translation. This means it can be finely tuned for specific tasks, offering flexibility in its application to particular translation needs.
5. **Performance on Specific Tasks**: Models like Arabart have demonstrated excellent performance on Arabic-specific tasks such as automatic translation, summarization, and sentiment detection. This suggests they can offer similar benefits in your translation exercise.



In [ ]:
!nvidia-smi

Thu Mar 14 00:58:31 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   56C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!git clone https://github.com/huggingface/transformers.git
!pip install git+https://github.com/huggingface/transformers.git
!pip install datasets
!pip install evaluate
!pip install sentencepiece
!pip install tensorboardX
!pip install accelerate -U
!pip install sacrebleu

Cloning into 'transformers'...
remote: Enumerating objects: 191301, done.
remote: Counting objects: 100% (191235/191235), done.
remote: Compressing objects: 100% (44857/44857), done.
remote: Total 191301 (delta 136280), reused 189480 (delta 135041), pack-reused 66
Receiving objects: 100% (191301/191301), 199.29 MiB | 26.19 MiB/s, done.
Resolving deltas: 100% (136280/136280), done.
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-n032zz0h
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-n032zz0h
  Resolved https://github.com/huggingface/transformers.git to commit f738ab3b5d30e30c43a4c3d00ca8939f8a4d4427
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.39.0.dev0-py3-none-any.whl size=8680419 sha256=dfa88622058f461cbdbe99a7acd57b788f74eda92bf7f2

In [ ]:
!(python3 transformers/examples/pytorch/translation/run_translation.py \
    --model_name_or_path moussaKam/arabart \
    --do_train \
    --do_eval \
    --source_lang Darija \
    --target_lang MSA \
    --train_file ./train2.json \
    --validation_file ./test-4000.json \
    --output_dir ./tmp/tst-translation \
    --per_device_train_batch_size=32 \
    --per_device_eval_batch_size=32 \
    --save_steps=200 --eval_steps=200 \
    --logging_steps=200 \
    --save_total_limit=15 \
    --overwrite_output_dir \
    --push_to_hub=False \
    --preprocessing_num_workers=8 \
    --dataloader_num_workers=2 \
    --dataloader_prefetch_factor=2 \
    --num_train_epochs=5 \
    --evaluation_strategy='steps' \
    --save_strategy='steps' \
    --load_best_model_at_end=True \
    --predict_with_generate)

2024-03-14 02:10:57.510546: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-14 02:10:57.510605: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-14 02:10:57.512042: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-14 02:10:58.667394: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
03/14/2024 02:11:00 - WARNING - __main__ - Process rank: 0, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
03/14/2024 02:11:00 - INFO - __main__ - Training/evaluation parameters Seq2SeqTrainingArguments(
_n_gp